<a href="https://colab.research.google.com/github/Viny2030/HUMAI/blob/main/ejercicios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div align="center">
<a href="https://colab.research.google.com/github/institutohumai/machine_learning/blob/main/8_ShapyAnomalias/ejercicios/ejercicios.ipynb"> <img src='https://colab.research.google.com/assets/colab-badge.svg' /> </a>
 Recordá abrir en una nueva pestaña </div>



# Detección de anomalías

En ciencia de datos, la *detección de anomalías* es la identificación de observaciones raras, que se desvían significativamente de la "normalidad" definida en un conjunto de datos.

![](https://i.imgur.com/vTIXxm8m.jpg)


Entendida como **un paso en el análisis exploratorio de datos o EDA**, podemos buscar anomalías considerándolas ruido en los datos que puede ensuciar la muestra y deteriorar la capacidad de un clasificador o un regresor. En estos escenarios hablamos de detección y limpieza de _outliers_. En la clase teórica y la notebook asociada podrán ver técnicas relacionadas con esto como las Tukey's fences.

Pero la detección de anomalías es a veces también **un fin en sí misma**. En estos casos, no consideramos "ruido" a los datos atípicos ni buscamos desecharlos: al contrario, identificarlos resulta del mayor valor. Por ejemplo, en un entorno médico podría ser de interés detectar muestras de sangre o imágenes anómalas. Otro ejemplo podría ser los sensores de una turbina de un avión: la detección de una anomalía podría ser de enorme relevancia en estos casos.

Si bien existen otros enfoques, la detección de anomalías suele encararse como un problema de **Aprendizaje Automático No-Supervisado**. En este enfoque, se ajusta un modelo a los datos, pero no hay etiquetas `y`, solamente la `X`. Esto se debe a que, por definición, las anomalías son escasas y no es fácil conseguir un dataset con muchas de ellas.


Existen diferentes formas de ajustar modelos. Podemos pensar que se busca construir una cierta idea de "normalidad" a partir de los datos para, desde esa idea, identificar los datos que se apartan de ella como "anomalías".


# Detección de anomalías en un dataset de cáncer de mama

Vamos a usar un dataset público de mamografías bajado de [ODDS](http://odds.cs.stonybrook.edu/mammography-dataset/). Este consiste de `11183` mamografías con `6` caracteristicas (anonimizadas). De estas, solo `260` corresponden a tumores malignos.

Entrenaremos un modelo de detección de anomalías sin considerar la etiqueta y veremos cuán bien funciona esta detección de anomalías para identificar tumores malignos.

In [1]:
# Importamos las librerías relevantes
import numpy as np
import pandas as pd
import warnings; warnings.simplefilter('ignore')

from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, recall_score

In [2]:
try:
    df = pd.read_csv("odds-mammografia-cancer-de-mama.csv")
except:
    df = pd.read_csv("https://storage.googleapis.com/humai-datasets/aws_s3/ml_avanzado/3_Deteccion_de_anomalias_y_shap_values/ejercicios/odds-mammografia-cancer-de-mama.csv")
    df.to_csv("odds-mammografia-cancer-de-mama.csv", index=False)

In [3]:
# 6 features anonimizados
print(df.shape)
df.head()

(11183, 7)


,attr_0,attr_1,attr_2,attr_3,attr_4,attr_5,es_cancer
0,0.230020,5.072578,-0.276061,0.832444,-0.377866,0.480322,0
1,0.155491,-0.169390,0.670652,-0.859553,-0.377866,-0.945723,0
2,-0.784415,-0.443654,5.674705,-0.859553,-0.377866,-0.945723,0
3,0.546088,0.131415,-0.456387,-0.859553,-0.377866,-0.945723,0
4,-0.102987,-0.394994,-0.140816,0.979703,-0.377866,1.013566,0


In [4]:
# Solo 260 datos son tumores malignos
df['es_cancer'].value_counts()

,count
es_cancer,
0,10923
1,260


In [5]:
# Corresponde al 2.3% de los datos
df['es_cancer'].value_counts(normalize=True)

,proportion
es_cancer,
0,0.97675
1,0.02325


## Entrenamos un modelo [`IsolationForest`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.IsolationForest.html#sklearn.ensemble.IsolationForest)

Isolation Forest es un algoritmo de detección de anomalias. Este se basa en construir un árbol de decisión sobre los datos (varios, en realidad) y medir cuantos cortes fueron necesarios para _aislar_ un punto (de ahí su nombre).
Esta técnica se diferencia de la mayoría de los modelos de detección de anomalías que construyen una representación de "normalidad" para luego ver cuánto se aparta un punto de esta.

<img src='https://i.imgur.com/LkIceyM.png' style="width:800px;height:300px">

Separemos el set de datos en datos de entrenamiento y validación de la forma usual:

In [6]:
X, y = df.drop('es_cancer',axis=1), df['es_cancer']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=1)
contamination = y_train.value_counts(normalize=True)[1]
contamination

0.023250268272326218

In [8]:
y_test.value_counts()

,count
es_cancer,
0,2731
1,65


In [9]:
model = IsolationForest(contamination='auto', random_state=1)

In [10]:
# El método fit no usa y_train!! Está aprendiendo a detectar anomalías, no etiquetas o clases
model.fit(X_train)

IsolationForest(random_state=1)

El método `predict` de un modelo de detección de anomalías retorna `1` (outlier) o `-1` (normal):

```python
Signature: model.predict(X)
Docstring:
Predict if a particular sample is an outlier or not.

Parameters
----------
X : {array-like, sparse matrix} of shape (n_samples, n_features)
    The input samples. Internally, it will be converted to
    ``dtype=np.float32`` and if a sparse matrix is provided
    to a sparse ``csr_matrix``.

Returns
-------
is_inlier : ndarray of shape (n_samples,)
    For each observation, tells whether or not (+1 or -1) it should
    be considered as an inlier according to the fitted model.
```

In [ ]:
y_pred = model.predict(X_test)
y_pred = [1 if y == -1 else 0 for y in y_pred]

Veamos la matriz de confusión y el reporte de clasificación para este detector:

In [ ]:
cm = confusion_matrix(y_test, y_pred)
cm = pd.DataFrame(cm, columns=['Pred=Sano', 'Pred=Cancer'], index=['Actual=Sano', 'Actual=Cancer'])
cm

,Pred=Sano,Pred=Cancer
Actual=Sano,2401,330
Actual=Cancer,22,43


In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      0.88      0.93      2731
           1       0.12      0.66      0.20        65

    accuracy                           0.87      2796
   macro avg       0.55      0.77      0.56      2796
weighted avg       0.97      0.87      0.91      2796



Obtuvimos un `66%` de exhaustividad en identificación de muestras cancerosas con este sencillo detector de anomalías.

Veremos la tasa de falsos positivos, que funciona como un "trade-off" con la exhaustividad. Esto es, podemos aumentar la exhaustividad aumentando la tasa de falsos positivos.


In [ ]:
recall_score(y_test, y_pred)

0.6615384615384615

In [ ]:
# Ratio de Falsos Positivos False positive rate FPR
fp = cm.loc['Actual=Sano', 'Pred=Cancer']
p  = cm.loc['Actual=Sano'].sum()
fpr = fp / p
fpr

0.1208348590259978

La tasa de falsos positivos es de 12%.
Vamos a probar diferentes valores del parámetro "contamination" para ver cómo se mueven la exhaustividad y la tasa de falsos positivos, y ver si encontramos alguna combinación que nos guste más...

In [ ]:
def get_results_for_factor(X_train, y_train, X_test, y_test, contamination_factor, verbose=False):
    contamination = y_train.value_counts(normalize=True)[1]
    C = contamination_factor*contamination
    model = IsolationForest(contamination=C, random_state=1)
    model.fit(X_train)
    y_pred = model.predict(X_test)
    y_pred = [1 if y == -1 else 0 for y in y_pred]

    cm = confusion_matrix(y_test, y_pred)
    cm = pd.DataFrame(cm, columns=['Pred=Sano', 'Pred=Cancer'], index=['Actual=Sano', 'Actual=Cancer'])
    fp = cm.loc['Actual=Sano', 'Pred=Cancer']
    p  = cm.loc['Actual=Sano'].sum()
    fpr = fp / p
    recall = recall_score(y_test, y_pred)
    results = {'contamination_factor': contamination_factor, 'recall': round(recall, 2), 'fpr': round(fpr, 2)}

    if verbose:
        print(results)
        print(classification_report(y_test, y_pred))
        display(cm)
    return results

In [ ]:
all_res = []
for factor in range(1, 20):
    res = get_results_for_factor(X_train, y_train, X_test, y_test, factor)
    print(res)
    all_res.append(res)

{'contamination_factor': 1, 'recall': 0.22, 'fpr': 0.01}
{'contamination_factor': 2, 'recall': 0.32, 'fpr': 0.03}
{'contamination_factor': 3, 'recall': 0.49, 'fpr': 0.06}
{'contamination_factor': 4, 'recall': 0.54, 'fpr': 0.08}
{'contamination_factor': 5, 'recall': 0.62, 'fpr': 0.11}
{'contamination_factor': 6, 'recall': 0.72, 'fpr': 0.13}
{'contamination_factor': 7, 'recall': 0.8, 'fpr': 0.15}
{'contamination_factor': 8, 'recall': 0.82, 'fpr': 0.19}
{'contamination_factor': 9, 'recall': 0.82, 'fpr': 0.21}
{'contamination_factor': 10, 'recall': 0.83, 'fpr': 0.24}
{'contamination_factor': 11, 'recall': 0.86, 'fpr': 0.26}
{'contamination_factor': 12, 'recall': 0.91, 'fpr': 0.28}
{'contamination_factor': 13, 'recall': 0.92, 'fpr': 0.31}
{'contamination_factor': 14, 'recall': 0.92, 'fpr': 0.33}
{'contamination_factor': 15, 'recall': 0.92, 'fpr': 0.35}
{'contamination_factor': 16, 'recall': 0.92, 'fpr': 0.37}
{'contamination_factor': 17, 'recall': 0.92, 'fpr': 0.4}
{'contamination_factor': 

### 80% de exhaustividad (recall) con solo 15% de falsos positivos!!

In [ ]:
res = get_results_for_factor(X_train, y_train, X_test, y_test, 7, verbose=True)

{'contamination_factor': 7, 'recall': 0.8, 'fpr': 0.15}
              precision    recall  f1-score   support

           0       0.99      0.85      0.91      2731
           1       0.11      0.80      0.19        65

    accuracy                           0.84      2796
   macro avg       0.55      0.82      0.55      2796
weighted avg       0.97      0.84      0.90      2796



,Pred=Sano,Pred=Cancer
Actual=Sano,2310,421
Actual=Cancer,13,52


# Ejercicios:

1. Importar un nuevo modelo de Detección de Anomalías de sklearn
Por ejemplo:
```python
from sklearn.neighbors import LocalOutlierFactor
```

[Esta página de la documentación de sklearn](https://scikit-learn.org/stable/auto_examples/miscellaneous/plot_anomaly_comparison.html#sphx-glr-auto-examples-miscellaneous-plot-anomaly-comparison-py) puede ser de utilidad para aquellos que se den maña con el Inglés.

2. Hacer un split entrenatimento-validación y ajustar el modelo sobre los datos de entrenamiento. Revisar la documentación de sklearn para entender qué argumentos tiene el modelo.
3. Obtener predicciones de anomalías probables para los datos de validación
4. Comparar esas predicciones de anomalías con la etiqueta "es cancer" para los datos de validación (`y_test`) usando las métricas de clasificación binaria conocidas


5. _Chequeo de comprensión: ¿En qué se diferencia lo que estamos haciendo con una clasificación binaria supervisada? ¿Te parece que los modelos de detección de anomalías no supervisados deberían funcionar mejor o peor que un clasificador binario para identificar muestras que corresponden a tumores malignos? ¿Por qué lo pensás?_


### Opcional - para el hogar
6. Intentar superar los resultados obtenidos por `IsolationForest` previamente (considerando el `Recall` y el `FPR`) con cualquier estrategía (buscar otros modelos de sklearn, argumentos de los constructores, etc).
